# Week 8 - Clustering (Unsupervised Learning)

## Learning Objectives
+ Hierarchical Clustering
+ K-Means clustering
    + Difference with hierarchical clustering
    + Reducing the dimensions of high dimensional data
        + PCA
        + t-SNE
    + Methods of finding number of clusters:
        + Using silhoutte analysis
        + Using elbow method

The contents of this tutorial are based on the chapter 10 of "An Introduction to Statistical Learning" by James G. et. al., python [notebook](https://nbviewer.jupyter.org/github/JWarmenhoven/ISL-python/blob/master/Notebooks/Chapter%2010.ipynb#Lab-3:-NCI60-Data-Example) on same chapter, sklearn [tutorial](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py) on k-means, sklearn [tutorial](https://scikit-learn.org/stable/modules/clustering.html) on clustering, sklearn [tutorial](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html) on agglomerative clustering, and introduction to dimensionality reduction [tutorial](https://www.datacamp.com/community/tutorials/introduction-t-sne).

## Loading the NCI-60 dataset

NCI60 is cancer cell line microarray data, which consists of 6,830 gene expression measurements on 64 cancer cell lines. So, our data is 64x6830, implying that the n<\<p. In case of such high-dimensional data, methods such as regression do not remain applicable. Dimensionality reduction is also useful for such data, before applying different ML models. 

Gene data is usually always a high-dimensionality data, and is useful in precision medicine - i.e. to recommend personalized treatment to patients. A common approach for such personalized treatment is categorizing the patients into various sub-types. For such sub-typing, clustering is usually performed and can be used to identify patients who are similar - to analyze their treatments and decide the further course of treatment for the patient under consideration.  

In our data, though we have cancer type label for each cell-line, in real-life data, we typically do not have this for all the patients, making the clustering quite important. Usually clustering methods are then analyzed on various datasets, and validation from the domain - such as the cancer types in our case are used to identifying a more superior technique to be used. 

Our data has gene expression measurement which is usually achieved by quantifying levels of the gene product, which is often a protein.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
rng = np.random.RandomState(10)

In [ ]:
df = pd.read_csv('https://web.stanford.edu/~hastie/ElemStatLearn/datasets/nci.data.csv', skiprows=1, header=None).T
# from pandas 0.19.2 onwards, we can directly pass the url to read_csv

In [ ]:
df.head()

In [ ]:

                            # Set new column names
                            # Drop duplicated row

In [ ]:
labs = pd.read_csv('https://web.stanford.edu/~hastie/ElemStatLearn/datasets/nci.label.txt', names=['type'], dtype="category")

In [ ]:
labs.type = labs.type.str.strip()

In [ ]:
labs.head()

In [ ]:
labs.value_counts()

In [ ]:
labs.nunique()

# Hierarchical Clustering

Let us first try to find out whether or not the observations cluster into distinct types of cancer. 

Here we have an option to standardize the variables to have mean zero and standard deviation one. However, this step is optional and should be performed only if we want each gene to be on the same scale. We could reasonably argue that it is better not to scale genes. However, it is a choice to make as a data analyst, and also evaluate experimentally.

## Agglomerative Clustering
This is a bottom-up clustering, and is built starting from the leaves and combining clusters up to the trunk. Each dendogram will represent each of our 64 samples. As we move up the tree, some leaves begin to fuse into branches. These correspond to
observations that are similar to each other. As we move higher up the tree, branches themselves fuse, either with leaves or other branches. The earlier (lower in the tree) fusions occur, the more similar the groups of observations are to each other. Thus, the height of the fusion depicts how different the observations are.

![Agglomerative Clustering](https://dashee87.github.io/images/hierarch.gif)

The [```AgglomerativeClustering```](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) class in sklearn implements four different types of linkages. The linkage criterion determines which distance to use between sets of observation. The algorithm will merge the pairs of cluster that minimize this criterion.
+ ‘ward’ minimizes the variance of the clusters being merged.
+ ‘average’ uses the average of the distances of each observation of the two sets.
+ ‘complete’ or ‘maximum’ linkage uses the maximum distances between all observations of the two sets.
+ ‘single’ uses the minimum of the distances between all observations of the two sets.

The default is ward. Usually complete linkage is used for Gene data.

In [ ]:
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
%matplotlib inline

Let us write a function to plot dendogram. We use the [```cluster.hierarchy.dendogram```](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html#scipy.cluster.hierarchy.dendrogram). The [```cluster.hierarchy```]() is available in scipy which has functions that cut hierarchical clusterings into flat clusterings or find the roots of the forest formed by a cut by providing the flat cluster ids of each observation. We will see usage of both these API - the sklearn and the scipy.

In [ ]:
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)

model = model.fit(df)
model.labels_

In [ ]:
plt.figure(figsize=(10,10))
plt.title('Hierarchical Clustering Dendrogram')
# plot the top three levels of the dendrogram, where p is the number of levels to truncate at
# how do we use plot_dendrogram?

plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

The ```model.children_``` attribute we use shows the children of each non-leaf node. Values less than ```n_samples``` correspond to leaves of the tree which are the original samples. A node ```i``` greater than or equal to ```n_samples``` is a non-leaf node and has children ```children_[i - n_samples]```. Alternatively at the i-th iteration, ```children[i][0]``` and ```children[i][1]``` are merged to form node ```n_samples + i```.

In [ ]:
model.children_[:5,:] 

Now we can use the scipy API and also scale the gene data and see the clusters. 

In [ ]:
from scipy.cluster import hierarchy
from sklearn.preprocessing import scale

In [ ]:
X = pd.DataFrame(scale(df), index=labs.type, columns=df.columns)

We can scale data using ```scale(df)``` which centers the mean and component wise scale to unit variance, along any axis. We scale such that the mean of every column is 0 and the standard deviation of every column is 1.

In [ ]:
# how do we show that the data has been scaled?



In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(20,20))

for linkage, cluster, ax in zip([hierarchy.complete(X), hierarchy.average(X), hierarchy.single(X)],
                                ['c1','c2','c3'],
                                [ax1,ax2,ax3]):
    cluster = dendrogram(linkage, labels=X.index, orientation='right', color_threshold=0, leaf_font_size=10, ax=ax)

ax1.set_title('Complete Linkage')
ax2.set_title('Average Linkage')
ax3.set_title('Single Linkage');

We see that the choice of linkage affects the results obtained. Typically, single linkage will tend to yield trailing clusters: very large clusters onto which individual observations attach one-by-one. On the other hand, complete and average linkage tend to yield more balanced, attractive clusters. For this reason, complete and average linkage are generally preferred to single linkage. 

In the complete linkage, we can set the threshold at 140, to get four clusters. Increasing the threshold decreases the number of clusters. For example, increasing the threshold to 150 results in only 2 clusters. This is useful in selecting the number of clusters we want. 

In [ ]:
plt.figure(figsize=(10,20))
cut4 = hierarchy.dendrogram(hierarchy.complete(X),
                            labels=X.index, orientation='right', color_threshold=140, leaf_font_size=10)
# this plot is compatible with matplotlib. can we insert a line at the threshold of 140?

Based on this cut, we can see the four clusters in four colors. We see all the leukemia cell lines fall in one cluster,
while the breast cancer cell lines are spread out over three different clusters

In [ ]:
cut4c = hierarchy.dendrogram(hierarchy.complete(X), truncate_mode='lastp', p=4,
                             show_leaf_counts=True)

# K-Means for same number of clusters
We can perform K-Means with same number of clusters and check the clusters that are formed. 

In K Means clustering, since we start with random choice of clusters, the results produced by running the algorithm multiple times might differ. While results are reproducible in Hierarchical clustering.

![KMeans Clustering](https://miro.medium.com/max/600/1*h2WdqGZD6WsNcUdwZDqsFA.gif)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
np.random.seed(2)
# we set up KMeans clustering with 4 clusters, let the algorithm run with 50 different centroid seeds. 



In [ ]:
km4.labels_

In [ ]:
# Observations per KMeans cluster
pd.Series(km4.labels_).value_counts().sort_index()

We see that though clustering has been done in same number of clusters - the observations per cluster is different and the clusters themselves must also be different. You can do visualizations along with the cancer types to actually see the difference between the clusters using both algorithms.

# Finding k 
Unlike the hierarchical clustering, the k-means clustering has the problem of specifying the number of clusters (k). How do we find this? We have two popular methods for this approach:
+ Silhoutte Method
+ Elbow Method

## Silhoutte method
Silhouette analysis can be used to study the separation distance between the resulting clusters. The silhouette plot displays a measure of how close each point in one cluster is to points in the neighboring clusters and thus provides a way to assess parameters like number of clusters visually. This measure has a range of \[-1, 1\].

Silhouette coefficients (as these values are referred to as) near +1 indicate that the sample is far away from the neighboring clusters. A value of 0 indicates that the sample is on or very close to the decision boundary between two neighboring clusters and negative values indicate that those samples might have been assigned to the wrong cluster.

### Performing dimensionality reduction - PCA
Sometimes performing clustering on the first few principal component score vectors can give better results than performing clustering on the full data. In this situation, we might view the principal component step as one of denoising the data. You can ofcourse perform the clustering without the PCA, however, on this specific data, the PCA helps in finding more meaningful clusters using K-means. So, let us first perform PCA on the data.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(random_state=rng) # default number of features to keep is the same as number of samples

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pca.fit(df)


# how can we see how much of the variance is explained by the principal components?


In [ ]:
df2_plot = pd.DataFrame(pca.fit_transform(df))

In [ ]:
df2_plot.head()

In [ ]:
pd.DataFrame([list(df2_plot.iloc[:,:5].std(axis=0, ddof=0)),
              pca.explained_variance_ratio_[:5],
              np.cumsum(pca.explained_variance_ratio_[:5])],
             index=['Standard Deviation', 'Proportion of Variance', 'Cumulative Proportion'],
             columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])

In [ ]:
df2_plot.iloc[:,:10].var(axis=0, ddof=0).plot(kind='bar', rot=0) # ddof is delta degree of freedom, rot is rotation for the ticks 
plt.ylabel('Variances');

In [ ]:
plt.xlabel('pc1')
plt.ylabel('pc2')
sns.scatterplot(x=df2_plot.iloc[:,0],y=df2_plot.iloc[:,1]) # we plot the first two principal components

Now once we have the principal components, the 64 dimensional data can be used to perform silhoutte analysis. 

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm

In [ ]:
def silhoutte_plot(n_clusters): # from https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
    fig, ax = plt.subplots()

    # The silhouette coefficient can range from -1, 1, but ranges from -0.1 to 1 for our data
    ax.set_xlim([-0.1, 1])
    
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax.set_ylim([0, len(df2_plot) + (n_clusters + 1) * 10])

    clusterer = KMeans(n_clusters=n_clusters, n_init=50, random_state=rng)
    cluster_labels = clusterer.fit_predict(df2_plot)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(df2_plot, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(df2_plot, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples
        
    ax.set_title("The silhouette plot for the various clusters.")
    ax.set_xlabel("The silhouette coefficient values")
    ax.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax.axvline(x=silhouette_avg, color="red", linestyle="--")

    # Labeling the clusters
    centers = clusterer.cluster_centers_

    plt.title(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

A good number of clusters is such that (1) no cluster is below the average score and (2) the clusters as proportionate as possible in size.

In [ ]:
range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:
    silhoutte_plot(n_clusters)

From the silhoutte analysis, the k=3 and k=4 seem to have good clusters, as for both the cases, the clusters are proportionate in size, and also no cluster is below the average score. Let us visualize the results for k=4 then. 

In [ ]:
kmeans_pca = KMeans(n_clusters=4, n_init=100, max_iter=400, init='k-means++', random_state=rng).fit(df2_plot)
print('KMeans PCA Silhouette Score: {}'.format(silhouette_score(df2_plot, kmeans_pca.labels_, metric='euclidean')))
labels_pca = kmeans_pca.labels_
clusters_pca = pd.concat([df2_plot, pd.DataFrame({'pca_clusters':labels_pca})], axis=1)

In [ ]:
clusters_pca.pca_clusters.value_counts()

In [ ]:
sns.scatterplot(data=clusters_pca, x=clusters_pca.iloc[:,0], y=clusters_pca.iloc[:,1], hue='pca_clusters', palette="deep")

## Elbow Method
This is another method used for selecting the number of clusters. We fit the model with range of K values. If the line chart resembles an arm, then the “elbow” (the point of inflection on the curve) is a good indication that the underlying model fits best at that point. 

### Performing dimensionality reduction - tSNE
t-Distributed Stochastic Neighbor Embedding (t-SNE) is a non-linear technique for dimensionality reduction that is particularly well suited for the visualization of high-dimensional datasets. It is extensively applied in image processing, NLP, genomic data and speech processing. In simple terms, t-SNE minimizes the divergence between two distributions: a distribution that measures pairwise similarities of the input objects and a distribution that measures pairwise similarities of the corresponding low-dimensional points in the embedding. This is a particularly useful technique for visualizing high-dimensional data.

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
# how do we apply tsne and put the results in a dataframe?


In [ ]:
df3.shape

Let us visualize this two dimensional data. 

In [ ]:
plt.xlabel('tsne1')
plt.ylabel('tsne2')
sns.scatterplot(x=df3.iloc[:,0], y=df3.iloc[:,1])

To find the optimal number of clusters - we plot the sum of squared distances of samples to their closest cluster center, available as ```inertia_``` attribute of the K-Means class. 

In [ ]:
sse = []
k_list = range(1, 20)
for k in k_list:
    km = KMeans(n_clusters=k, random_state=rng)
    km.fit(df3)
    sse.append(km.inertia_)
    
tsne_results_scale = pd.DataFrame({'Cluster': range(1,20), 'SSE': sse})
tsne_results_scale.head()

In [ ]:
plt.figure(figsize=(10,8))
sns.lineplot(data=tsne_results_scale, x='Cluster', y='SSE', marker='o')
plt.xticks(range(1,20))
plt.title('Optimal Number of Clusters using Elbow Method (tSNE Data)')

We see that the elbow seems to be at k=4. Let us select 4 clusters and visualize. 

In [ ]:
kmeans_tsne_scale = KMeans(n_clusters=4, n_init=100, max_iter=400, init='k-means++', random_state=rng).fit(df3)
print('KMeans tSNE Scaled Silhouette Score: {}'.format(silhouette_score(df3, kmeans_tsne_scale.labels_, metric='euclidean')))
labels_tsne_scale = kmeans_tsne_scale.labels_
clusters_tsne_scale = pd.concat([df3, pd.DataFrame({'tsne_clusters':labels_tsne_scale})], axis=1)

In [ ]:
clusters_tsne_scale.head()

In [ ]:
clusters_tsne_scale.tsne_clusters.value_counts().sort_index()

In [ ]:
plt.xlabel('tsne1')
plt.ylabel('tsne2')
sns.scatterplot(data=clusters_tsne_scale, x='tsne1', y='tsne2', hue='tsne_clusters', palette='deep')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

A warning: t-SNE often fails to preserve the global geometry of the data. This means that the relative position of clusters on the t-SNE plot is almost arbitrary and depends on random initialisation more than on anything else. While this may not be a problem in some situations, scRNA-seq data sets often exhibit biologically meaningful hierarchical structure, e.g. encompass several very different cell classes, each further divided into various types. Typical t-SNE plots do not capture such global structure, yielding a suboptimal and potentially misleading visualisation. So, t-SNE may not always yield best dimensionality reduction for genetic data, however, it has shown very good results for high-dimensional data in general.

### How would we analyze which clustering is most representative for this type of data?
Usually the domain knowledge of the cancer types and which algorithm has given us good clusters which are most representative is evaluated. Another approach used for gene data is to evaluate the different patient characteristics, such as Age, co-morbidities, etc. to find if the clusters are actually identifying such patterns. Also, using multiple datasets is usually required in such unsupervised tasks. 